משימות:
1. לייצר פיצ'רים מטמטיים מהקטגוריות והתקצירים - בוצע
2.  לחבר אותם לאינדקסים של הטבלה לפי הקטגוריות - בוצע
3. ליצור קובץ שלהם- בוצע
4. להכניס אותם למודל כשכבה נפרדת, להבין כיצד נכון לשלב את זה - בוצע 
5. לחזות - בוצע
6. ליצור מודל "ברירת מחדל" שממליץ על פופולריות
7. לראות איך משבים ריקול ופרפ גם למודל הזה
8. לשקול לסנן קוראים עם פחות מ10 ספרים
9. להוסיף למצגת

In [4]:
import math, pydot, graphviz
import numpy as np
import pandas as pd
from collections import defaultdict
# Split the data into train and test sets
# importing relevant libraries
from sklearn.model_selection import train_test_split

In [5]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

In [2]:
#!pip install pydot

In [3]:
#!pip install graphviz

In [4]:
df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv'

In [5]:
df = pd.read_csv(r"C:\Users\yotam\Desktop\naya\Preprocessed_data.csv")

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country,isbn_num
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa,0
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada,1
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada,1
3,3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN,1
4,4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada,1


In [6]:
df = df[df['rating']!=0]
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])

In [7]:
#split to train andtest while keeping each user in the train and test data
train_users, test_users = train_test_split(df, test_size = 0.2 , random_state= 42)

In [8]:
df = df[df['rating']!=0]
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])
train_users, test_users = train_test_split(df, test_size = 0.2 , random_state= 42)
x_train = train_users.drop(columns=['rating'])
y_train = train_users["rating"]

x_test = test_users.drop(columns=['rating'])
y_test = test_users["rating"]
all_users = train_users["user_id"].unique()
all_books = train_users["isbn_num"].unique()

## Implemation of TenserFlow

In [9]:
all_users = train_users["user_id"].unique()
all_books = train_users["isbn_num"].unique()

In [9]:
# user pipeline
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users)+1, output_dim=32)(user_as_integer)

In [10]:
# book pipeline
book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books)+1, output_dim=32)(book_as_integer)

In [11]:
# dot product
dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
flatten = tf.keras.layers.Flatten()(dot)

In [12]:
# model input/output definition
model = tf.keras.Model(inputs=[user_input, book_input], outputs=flatten)

In [13]:
model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [14]:
## this model is quite bad with an avg of 3 starts in rating deviation
# model.fit(
#     x={
#         "user": x_train["user_id"],
#         "book": x_train["isbn_num"]
#     },
#     y=y_train.values,
#     batch_size=256,
#     epochs=100,
#     validation_split=0.1, # for early stopping
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True)
#     ],
# )


Epoch 1/100
2538/2538 [==============================] - 57s 22ms/step - loss: 22.8869 - mean_absolute_error: 2.8379 - val_loss: 23.1210 - val_mean_absolute_error: 2.8641
Epoch 2/100
2538/2538 [==============================] - 59s 23ms/step - loss: 22.8537 - mean_absolute_error: 2.8359 - val_loss: 23.1175 - val_mean_absolute_error: 2.8655
Epoch 3/100
  14/2538 [..............................] - ETA: 1:26 - loss: 22.9367 - mean_absolute_error: 2.8693

KeyboardInterrupt: 

# full pipline

In [42]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)

dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(flatten)

model = tf.keras.Model(inputs=[user_input, book_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [43]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 book (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 integer_lookup_27 (Integer  (None, 1)                    0         ['user[0][0]']                
 Lookup)                                                                                          
                                                                                                  
 integer_lookup_28 (Integer  (None, 1)                    0         ['book[0][0]']         

In [46]:
 model.fit(
     x={
         "user": x_train["user_id"],
         "book": x_train["isbn_num"]
     },
     y=y_train.values,
     batch_size=128,
     epochs=20,
     validation_split=0.1, # for early stopping
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
     ],
 )


Epoch 1/20
1164/1164 [==============================] - 11s 9ms/step - loss: 2.4961 - mean_absolute_error: 1.1847 - root_mean_squared_error: 1.5799 - val_loss: 3.8244 - val_mean_absolute_error: 1.5817 - val_root_mean_squared_error: 1.9556
Epoch 2/20
1164/1164 [==============================] - 11s 9ms/step - loss: 2.2753 - mean_absolute_error: 1.1112 - root_mean_squared_error: 1.5084 - val_loss: 3.7718 - val_mean_absolute_error: 1.5675 - val_root_mean_squared_error: 1.9421
Epoch 3/20
1164/1164 [==============================] - 11s 9ms/step - loss: 2.0740 - mean_absolute_error: 1.0415 - root_mean_squared_error: 1.4401 - val_loss: 3.7356 - val_mean_absolute_error: 1.5574 - val_root_mean_squared_error: 1.9328
Epoch 4/20
1164/1164 [==============================] - 11s 9ms/step - loss: 1.8901 - mean_absolute_error: 0.9757 - root_mean_squared_error: 1.3748 - val_loss: 3.7078 - val_mean_absolute_error: 1.5493 - val_root_mean_squared_error: 1.9256
Epoch 5/20
1164/1164 [======================

In [47]:
model.evaluate({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"]#.reshape(-1, 1)
},y_test)

1293/1293 [==============================] - 2s 1ms/step - loss: 3.6728 - mean_absolute_error: 1.5394 - root_mean_squared_error: 1.9164


[3.6727774143218994, 1.5394413471221924, 1.9164491891860962]

add user data

In [15]:
user_input = tf.keras.layers.Input(shape=(2,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)

dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(flatten)

model = tf.keras.Model(inputs=[user_input, book_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [16]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 2)]                  0         []                            
                                                                                                  
 book (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 integer_lookup_4 (IntegerL  (None, 2)                    0         ['user[0][0]']                
 ookup)                                                                                           
                                                                                                  
 integer_lookup_5 (IntegerL  (None, 1)                    0         ['book[0][0]']          

In [28]:
 model.fit(
     x={
         "user": x_train[["user_id",'age']],
         "book": x_train["isbn_num"]
     },
     y=y_train.values,
     batch_size=256,
     epochs=5,
     validation_split=0.1, # for early stopping
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
     ],
 )


Epoch 1/5
2901/2901 [==============================] - 153s 53ms/step - loss: 14.5075 - mean_absolute_error: 3.5297 - root_mean_squared_error: 3.8089 - val_loss: 14.3180 - val_mean_absolute_error: 3.4756 - val_root_mean_squared_error: 3.7839
Epoch 2/5
2901/2901 [==============================] - 150s 52ms/step - loss: 13.9449 - mean_absolute_error: 3.3842 - root_mean_squared_error: 3.7343 - val_loss: 14.1711 - val_mean_absolute_error: 3.3803 - val_root_mean_squared_error: 3.7644
Epoch 3/5
2901/2901 [==============================] - 156s 54ms/step - loss: 13.5081 - mean_absolute_error: 3.2392 - root_mean_squared_error: 3.6753 - val_loss: 14.2842 - val_mean_absolute_error: 3.3059 - val_root_mean_squared_error: 3.7794
Epoch 4/5
2901/2901 [==============================] - 158s 54ms/step - loss: 13.0820 - mean_absolute_error: 3.1004 - root_mean_squared_error: 3.6169 - val_loss: 14.4152 - val_mean_absolute_error: 3.2619 - val_root_mean_squared_error: 3.7967


 ### model with more inputs:

In [13]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)

feature_input = tf.keras.layers.Input(shape=(2,), name="user_book_data")
feature = tf.keras.layers.Dense(5, name="add_some_feature")(feature_input)


dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
con = tf.keras.layers.concatenate([flatten,feature])

squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(con)

model = tf.keras.Model(inputs=[user_input, book_input,feature_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [14]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 book (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 integer_lookup_11 (Integer  (None, 1)                    0         ['user[0][0]']                
 Lookup)                                                                                          
                                                                                                  
 integer_lookup_12 (Integer  (None, 1)                    0         ['book[0][0]']          

In [32]:
 model.fit(
     x={
         "user": x_train["user_id"],
         "book": x_train["isbn_num"],
         "user_book_data":x_train[['age','year_of_publication']]
     },
     y=y_train.values,
     batch_size=256,
     epochs=5,
     validation_split=0.1, # for early stopping
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
     ],
 )


Epoch 1/5
1080/1080 [==============================] - 40s 37ms/step - loss: 21.4861 - mean_absolute_error: 3.7821 - root_mean_squared_error: 4.6353 - val_loss: 21.3847 - val_mean_absolute_error: 3.7614 - val_root_mean_squared_error: 4.6244
Epoch 2/5
1080/1080 [==============================] - 33s 30ms/step - loss: 21.3023 - mean_absolute_error: 3.7455 - root_mean_squared_error: 4.6154 - val_loss: 21.2528 - val_mean_absolute_error: 3.7338 - val_root_mean_squared_error: 4.6101
Epoch 3/5
1080/1080 [==============================] - 35s 32ms/step - loss: 21.1866 - mean_absolute_error: 3.7205 - root_mean_squared_error: 4.6029 - val_loss: 21.1662 - val_mean_absolute_error: 3.7147 - val_root_mean_squared_error: 4.6007
Epoch 4/5
1080/1080 [==============================] - 35s 32ms/step - loss: 21.1061 - mean_absolute_error: 3.7021 - root_mean_squared_error: 4.5941 - val_loss: 21.1048 - val_mean_absolute_error: 3.7005 - val_root_mean_squared_error: 4.5940
Epoch 5/5
1080/1080 [===============

In [19]:
df.columns

Index(['Unnamed: 0', 'user_id', 'location', 'age', 'isbn', 'rating',
       'book_title', 'book_author', 'year_of_publication', 'publisher',
       'img_s', 'img_m', 'img_l', 'Summary', 'Language', 'Category', 'city',
       'state', 'country', 'isbn_num'],
      dtype='object')

In [14]:
pred = model.predict(x_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [21]:
pred = model.predict({
    "user": x_test[["user_id",'age']],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"]#.reshape(-1, 1)
})


6445/6445 [==============================] - 7s 1ms/step


In [29]:
model.evaluate({
    "user": x_test[["user_id",'age']],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"]#.reshape(-1, 1)
},y_test)

6445/6445 [==============================] - 9s 1ms/step - loss: 14.0976 - mean_absolute_error: 3.3696 - root_mean_squared_error: 3.7547


[14.097579002380371, 3.3696367740631104, 3.7546746730804443]

In [23]:
y_test

392141    0
120888    0
472434    0
206129    5
553117    0
         ..
580066    0
861273    0
666398    0
859941    0
667974    7
Name: rating, Length: 206235, dtype: int64

In [22]:
pred

array([[3.0711696, 3.445534 ],
       [2.91386  , 4.895115 ],
       [2.9571815, 3.5963862],
       ...,
       [2.9984808, 2.8038635],
       [3.0161195, 3.0132391],
       [2.9818997, 2.8458679]], dtype=float32)

In [22]:
import keras
import pydot
import pydotplus
from pydotplus import graphviz
from keras.utils import plot_model
from keras.utils import model_to_dot
keras.utils.pydot = pydot
#plot_model(model)

NameError: name 'model' is not defined

 ### model with emmbeding category and summerys:

In [3]:
def reader_encode(df,y=None):

    cat_var = ['Language','country']
    one_hot = OneHotEncoder(sparse=False)  # , drop = 'first')
    encoder_var_array = one_hot.fit_transform(df[cat_var])
    encoder_name = one_hot.get_feature_names_out(cat_var)
    encoder_vars_df = pd.DataFrame(encoder_var_array, columns=encoder_name)
    df = pd.concat([df, encoder_vars_df], axis=1)
    return df

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer


יש לנו בעיה בשימוש במשתנים האלה, יש הרבה מאוד ספרים שאין להם קטגוריה, אפשר להזין להם 0, אבל זה עדיין יהיה בעל משמעות מסוימת וזה לא יהיה נכון

In [6]:
df = pd.read_csv(r'C:\Users\yotam\Desktop\naya\df_clean_after_pca')
df = df[df['rating']!=0]
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])
df = df.reset_index(drop = True)

In [ ]:
df.info()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206820 entries, 0 to 206819
Columns: 251 entries, Unnamed: 0 to isbn_num
dtypes: float64(232), int64(5), object(14)
memory usage: 396.1+ MB


In [14]:
len(encoder_var_array)

206820

In [9]:
#from book_funcs import category_cleaner,df_cleaner, category_compliter, zero_droper, category_embedding

df =reader_encode(df)

train_users, test_users = train_test_split(df, test_size = 0.2 , random_state= 42)
x_train = train_users.drop(columns=['rating'])
y_train = train_users["rating"]

x_test = test_users.drop(columns=['rating'])
y_test = test_users["rating"]
all_users = train_users["user_id"].unique()
all_books = train_users["isbn_num"].unique()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206820 entries, 0 to 206819
Columns: 427 entries, Unnamed: 0 to country_nan
dtypes: float64(408), int64(5), object(14)
memory usage: 673.8+ MB


In [48]:
list(df.columns)

['Unnamed: 0',
 'index',
 'user_id',
 'location',
 'age',
 'isbn',
 'rating',
 'book_title',
 'book_author',
 'year_of_publication',
 'publisher',
 'img_s',
 'img_m',
 'img_l',
 'Summary',
 'Language',
 'Category',
 'city',
 'state',
 'country',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',


In [33]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)




feature_input = tf.keras.layers.Input(shape=(230,), name="user_book_data")
feature = tf.keras.layers.Dense(200, name="add_some_feature")(feature_input)
dropout1 = tf.keras.layers.Dropout(0.3, name="drop_some_feature")(feature)
feature2 = tf.keras.layers.Dense(64, activation = 'relu', name="add_some_feature2")(dropout1)

dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
con = tf.keras.layers.concatenate([flatten,feature2])
dense2 = tf.keras.layers.Dense(128, activation = 'relu', name="dence_layer2")(con)
dropout2 = tf.keras.layers.Dropout(0.3, name="drop_some_feature3")(dense2)
dense3 = tf.keras.layers.Dense(64, activation = 'relu', name="dence_layer3")(dropout2)
dropout3 = tf.keras.layers.Dropout(0.3, name="drop_some_feature4")(dense3)
dense4 = tf.keras.layers.Dense(16, activation = 'relu', name="dence_layer4")(dropout3)
dropout4 = tf.keras.layers.Dropout(0.3, name="drop_some_feature5")(dense4)

squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(dropout4)


#dropout2 = tf.keras.layers.Dropout(0.25, name="drop_some_feature2")(dropout2)



#dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
#add = tf.keras.layers.Add()([dot, user_bias, book_bias])
#flatten = tf.keras.layers.Flatten()(add)
#con = tf.keras.layers.concatenate([flatten,dropout2])
#dense2 = tf.keras.layers.Dense(100, activation = 'relu', name="dence_layer")(con)
#dropout2 = tf.keras.layers.Dropout(0.2, name="drop_some_feature3")(dense2)

#squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(con)

model = tf.keras.Model(inputs=[user_input, book_input,feature_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [34]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 book (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 integer_lookup_4 (IntegerL  (None, 1)                    0         ['user[0][0]']                
 ookup)                                                                                           
                                                                                                  
 integer_lookup_5 (IntegerL  (None, 1)                    0         ['book[0][0]']          

In [35]:
 model.fit(
     x={
         "user": x_train["user_id"],
         "book": x_train["isbn_num"],
         "user_book_data":x_train[[str(i) for i in range(0,230)]]
     },
     y=y_train.values,
     batch_size=128,
     epochs=30,
     validation_split=0.1, # for early stopping
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)
     ],
 )


Epoch 1/30
1164/1164 [==============================] - 23s 16ms/step - loss: 4.6601 - mean_absolute_error: 1.7444 - root_mean_squared_error: 2.1587 - val_loss: 3.1517 - val_mean_absolute_error: 1.4580 - val_root_mean_squared_error: 1.7753
Epoch 2/30
1164/1164 [==============================] - 12s 11ms/step - loss: 4.2731 - mean_absolute_error: 1.6274 - root_mean_squared_error: 2.0671 - val_loss: 2.9671 - val_mean_absolute_error: 1.3936 - val_root_mean_squared_error: 1.7225
Epoch 3/30
1164/1164 [==============================] - 13s 11ms/step - loss: 3.9745 - mean_absolute_error: 1.5246 - root_mean_squared_error: 1.9936 - val_loss: 2.9886 - val_mean_absolute_error: 1.3944 - val_root_mean_squared_error: 1.7288
Epoch 4/30
1164/1164 [==============================] - 13s 11ms/step - loss: 3.6755 - mean_absolute_error: 1.4162 - root_mean_squared_error: 1.9171 - val_loss: 3.0045 - val_mean_absolute_error: 1.3808 - val_root_mean_squared_error: 1.7333
Epoch 5/30
1164/1164 [==================

In [69]:
model.save('deep_model_emmbeding.h5')


C:\Users\yotam\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [36]:
model.evaluate({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"],#.reshape(-1, 1),
    "user_book_data":x_test[[str(i) for i in range(0,230)]]
},y_test)

1293/1293 [==============================] - 3s 2ms/step - loss: 3.0212 - mean_absolute_error: 1.4065 - root_mean_squared_error: 1.7382


[3.021188497543335, 1.4065338373184204, 1.7381566762924194]

In [54]:
pred = model.predict({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"],#.reshape(-1, 1),
    "user_book_data":x_test[[str(i) for i in range(0,230)]]
})


1293/1293 [==============================] - 2s 1ms/step


In [60]:
len(y_test)

41364

In [68]:
minp = 99
maxp =0
for i in range(len(pred)):
    for j in range(len(pred[i])):
        minp = min(minp,pred[i][j])
        maxp = max(maxp,pred[i][j])
print(minp,",",maxp)

5.5 , 9.991598


In [72]:
list(df.columns)

['Unnamed: 0',
 'index',
 'user_id',
 'location',
 'age',
 'isbn',
 'rating',
 'book_title',
 'book_author',
 'year_of_publication',
 'publisher',
 'img_s',
 'img_m',
 'img_l',
 'Summary',
 'Language',
 'Category',
 'city',
 'state',
 'country',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',


In [81]:
len(df['country'].unique())

166

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [98]:
len(list(df.columns)[251:])

176

In [122]:
df.info()

### with country and lang

In [20]:
list(x_train.columns)[249]

'isbn_num'

In [56]:
user_input = tf.keras.layers.Input(shape=(1,), name="user")
user_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_users)(user_input)
user_embedding = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=32)(user_as_integer)
user_bias = tf.keras.layers.Embedding(input_dim=len(all_users) + 1, output_dim=1)(user_as_integer)

book_input = tf.keras.layers.Input(shape=(1,), name="book")
book_as_integer = tf.keras.layers.IntegerLookup(vocabulary=all_books)(book_input)
book_embedding = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=32)(book_as_integer)
book_bias = tf.keras.layers.Embedding(input_dim=len(all_books) + 1, output_dim=1)(book_as_integer)




feature_input = tf.keras.layers.Input(shape=(230,), name="book_data")
feature = tf.keras.layers.Dense(200, name="add_some_feature")(feature_input)
dropout1 = tf.keras.layers.Dropout(0.3, name="drop_some_feature")(feature)
feature2 = tf.keras.layers.Dense(64, activation = 'relu', name="add_some_feature2")(dropout1)

user_feature_input = tf.keras.layers.Input(shape=(176,), name="user_data")
feature_u = tf.keras.layers.Dense(128, name="add_some_feature_u")(user_feature_input)
dropout1_u = tf.keras.layers.Dropout(0.4, name="drop_some_feature_u")(feature_u)
feature2_u = tf.keras.layers.Dense(64, activation = 'relu', name="add_some_feature2_u")(dropout1_u)


dot = tf.keras.layers.Dot(axes=2)([user_embedding, book_embedding])
add = tf.keras.layers.Add()([dot, user_bias, book_bias])
flatten = tf.keras.layers.Flatten()(add)
con_book_users = tf.keras.layers.concatenate([feature2,feature2_u])
feature2_u = tf.keras.layers.Dense(128, activation = 'relu', name="reader_book_type")(con_book_users)

con = tf.keras.layers.concatenate([flatten,feature2_u])#,feature2_u])
dense2 = tf.keras.layers.Dense(128, activation = 'relu', name="dence_layer2")(con)
dropout2 = tf.keras.layers.Dropout(0.3, name="drop_some_feature3")(dense2)
dense3 = tf.keras.layers.Dense(64, activation = 'relu', name="dence_layer3")(dropout2)
dropout3 = tf.keras.layers.Dropout(0.3, name="drop_some_feature4")(dense3)
dense4 = tf.keras.layers.Dense(16, activation = 'relu', name="dence_layer4")(dropout3)
dropout4 = tf.keras.layers.Dropout(0.3, name="drop_some_feature5")(dense4)

squash = tf.keras.layers.Lambda(lambda x: 9 * tf.nn.sigmoid(x) + 1)(dropout4)



model = tf.keras.Model(inputs=[user_input, book_input,feature_input,user_feature_input], outputs=squash)


model.compile(loss="mse", metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.RootMeanSquaredError()])


In [57]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 book (InputLayer)           [(None, 1)]                  0         []                            
                                                                                                  
 book_data (InputLayer)      [(None, 230)]                0         []                            
                                                                                                  
 user_data (InputLayer)      [(None, 176)]                0         []                            
                                                                                            

['Language_9',
 'Language_da',
 'Language_de',
 'Language_en',
 'Language_es',
 'Language_fr',
 'Language_it',
 'Language_nl',
 'Language_ru',
 'Language_th',
 'country_,',
 'country_aberdeenshire, united kingdom',
 'country_albania',
 'country_alderney',
 'country_america',
 'country_andalucia, spain',
 'country_antarctica',
 'country_argentina',
 'country_aruba',
 'country_australia',
 'country_austria',
 'country_bahamas',
 'country_bahrain',
 'country_barbados',
 'country_belgium',
 'country_belize',
 'country_bermuda',
 'country_bolivia',
 'country_brazil',
 'country_british columbia, canada',
 'country_bulgaria',
 'country_burma',
 'country_california, usa',
 'country_cambridgeshire, united kingdom',
 'country_canada',
 'country_canary islands, spain',
 'country_cape verde',
 'country_catalonia',
 'country_catalunya, spain',
 'country_cayman islands',
 'country_chile',
 'country_china',
 'country_co. carlow, ireland',
 'country_costa rica',
 'country_cuba',
 'country_cyprus',
 'c

In [58]:
 model.fit(
     x={
         "user": x_train["user_id"],
         "book": x_train["isbn_num"],
         "book_data":x_train[[str(i) for i in range(0,230)]], #category and summary
         'user_data':x_train[list(x_train.columns)[250:]]#+['age','year_of_publication']] #
     },
     y=y_train.values,
     batch_size=128,
     epochs=100,
     validation_split=0.1, # for early stopping
     callbacks=[
         tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True)
     ],
 )


Epoch 1/100
1164/1164 [==============================] - 68s 12ms/step - loss: 4.6662 - mean_absolute_error: 1.7448 - root_mean_squared_error: 2.1601 - val_loss: 3.0706 - val_mean_absolute_error: 1.4288 - val_root_mean_squared_error: 1.7523
Epoch 2/100
1164/1164 [==============================] - 13s 11ms/step - loss: 4.2700 - mean_absolute_error: 1.6256 - root_mean_squared_error: 2.0664 - val_loss: 2.9878 - val_mean_absolute_error: 1.4011 - val_root_mean_squared_error: 1.7285
Epoch 3/100
1164/1164 [==============================] - 13s 12ms/step - loss: 3.9728 - mean_absolute_error: 1.5228 - root_mean_squared_error: 1.9932 - val_loss: 3.0213 - val_mean_absolute_error: 1.4044 - val_root_mean_squared_error: 1.7382
Epoch 4/100
1164/1164 [==============================] - 14s 12ms/step - loss: 3.6649 - mean_absolute_error: 1.4135 - root_mean_squared_error: 1.9144 - val_loss: 2.9244 - val_mean_absolute_error: 1.3596 - val_root_mean_squared_error: 1.7101
Epoch 5/100
1164/1164 [=============

In [60]:
model.evaluate({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"],#.reshape(-1, 1),
    "book_data":x_test[[str(i) for i in range(0,230)]],
    'user_data':x_test[list(x_train.columns)[250:]]
},y_test)

1293/1293 [==============================] - 2s 2ms/step - loss: 2.9694 - mean_absolute_error: 1.3708 - root_mean_squared_error: 1.7232


[2.969360589981079, 1.3707752227783203, 1.723183274269104]

In [28]:
import numpy as np
from keras import Input, Model
from keras.layers import SimpleRNN, Dense


In [32]:
model.save('deep_model_emmbeding_leng.h5')


C:\Users\yotam\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
model.evaluate({
    "user": x_test["user_id"],#.reshape(-1, 1), # fill user 1 in many times
    "book": x_test["isbn_num"],#.reshape(-1, 1),
    "user_book_data":x_test[[str(i) for i in range(0,230)]],
    'user_data_add':x_test[list(x_train.columns)[250:]]
},y_test)

1293/1293 [==============================] - 3s 2ms/step - loss: 3.0913 - mean_absolute_error: 1.4042 - root_mean_squared_error: 1.7582


[3.0912551879882812, 1.404232144355774, 1.7581965923309326]

In [62]:
from torch.datasets.utils import Batch
from torch.sparse.jagged_tensor import KeyedJaggedTensor

ModuleNotFoundError: No module named 'torch.datasets'

In [89]:
len(df['img_s'].unique())

30918

In [92]:
len(df[['img_s']][df['img_s']=='9'])#[df['Summary']==9])


0

In [91]:
df.head(20)

,Unnamed: 0,index,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,...,"country_usa, kansas, usa",country_van wert,country_venezuela,country_vietnam,"country_virginia, usa",country_wales,"country_west indies, tobago","country_west yorkshire, united kingdom",country_ysa,country_nan
0,0,0,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4,4,67544,"toronto, ontario, canada",30.0000,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,7,116866,"ottawa, ,",34.7439,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,9,9,200273,"comber, ontario, canada",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,10,10,210926,"guelph, ontario, canada",34.7439,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,11,11,219008,"halifax, nova scotia, canada",60.0000,0002005018,7,Clara Callan,Richard Bruce Wright,2001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,12,12,263325,"fredericton, new brunswick, canada",27.0000,0002005018,6,Clara Callan,Richard Bruce Wright,2001.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,14,14,35704,"kansas city, missouri, usa",53.0000,0374157065,6,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,16,16,110912,"milpitas, california, usa",36.0000,0374157065,10,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
